## Web scrapping for reviews

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import requests
from bs4 import BeautifulSoup as bs

url = "https://www.amazon.in/Bajaj-Torque-New-Honeycomb-Technology/dp/B09R3QNGW5/ref=sr_1_1_sspa?crid=1DXZ4HMCXU7FH&keywords=bajaj+px+97+torque+room+36l+air+cooler&qid=1679637791&sprefix=%2Caps%2C200&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"
response = requests.get(url)

soup = bs(response.content,"html.parser") # creating soup object to iterate over the extracted content

In [3]:
wm_title=[]  
wm_date = []
wm_content = []
wm_rating = []

for i in range(1,150):

  link = "https://www.amazon.in/Bajaj-Torque-New-Honeycomb-Technology/dp/B09R3QNGW5/ref=sr_1_1_sspa?crid=1DXZ4HMCXU7FH&keywords=bajaj+px+97+torque+room+36l+air+cooler&qid=1679637791&sprefix=%2Caps%2C200&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1"
  soup = bs(response.content,"html.parser")# creating soup object to iterate over the extracted content 

  # extracting Review Title
  title = soup.find_all('a',class_='review-title-content')
  review_title = []
  for i in range(0,len(title)):
      review_title.append(title[i].get_text())
  review_title[:] = [titles.lstrip('\n') for titles in review_title]
  review_title[:] = [titles.rstrip('\n') for titles in review_title]
  wm_title = wm_title + review_title

  ## Extracting Ratings
  rating = soup.find_all('i',class_='review-rating')
  review_rating = []
  for i in range(2,len(rating)):
      review_rating.append(rating[i].get_text())
  #review_rating.pop(0)
  #review_rating.pop(0)
  review_rating[:] = [reviews.rstrip(' out of 5 stars') for reviews in review_rating]
  wm_rating = wm_rating + review_rating

  #Extracting Content of review
  review = soup.find_all("span",{"data-hook":"review-body"})
  review_content = []
  for i in range(0,len(review)):
      review_content.append(review[i].get_text())
  review_content[:] = [reviews.lstrip('\n') for reviews in review_content]
  review_content[:] = [reviews.rstrip('\n') for reviews in review_content]
  wm_content = wm_content + review_content

  #Extracting dates of reviews
  dates = soup.find_all('span',class_='review-date')
  review_dates = []
  for i in range(2,len(rating)):
      review_dates.append(dates[i].get_text())
  review_dates[:] = [reviews.lstrip('Reviewed in India on') for reviews in review_dates]
  #review_dates.pop(0)
  #review_dates.pop(0)
  wm_date  = wm_date + review_dates
  

In [4]:
print(len(wm_title))
print(len(wm_rating))
print(len(wm_content))
print(len(wm_date))

1192
894
1192
894


In [5]:
import pandas as pd

# check the lengths of the lists
print(len(wm_title))
print(len(wm_rating))
print(len(wm_content))
print(len(wm_date))

# assign NaN to the missing ratings
if len(wm_rating) < len(wm_title):
    diff = len(wm_title) - len(wm_rating)
    wm_rating += [float('nan')] * diff


1192
894
1192
894


In [7]:
import pandas as pd

# create DataFrame
df = pd.DataFrame()

# check lengths and adjust if needed
print(len(wm_title), len(wm_rating), len(wm_content), len(wm_date))
if len(wm_title) != len(wm_rating) or len(wm_rating) != len(wm_content):
    raise ValueError('Lengths of wm_title, wm_rating, and wm_content do not match')
elif len(wm_date) < len(wm_title):
    wm_date += [None] * (len(wm_title) - len(wm_date))

# add columns to DataFrame
df['Title'] = wm_title
df['Ratings'] = wm_rating
df['Comments'] = wm_content
df['Date'] = wm_date

# print the first 5 rows of the DataFrame
print(df.head())

1192 1192 1192 894
                                               Title Ratings  \
0  Platina on, Garmi gone! Efficient cooler for n...     4.0   
1                                        Noise level     4.0   
2                                   Worthy for value     4.0   
3          Im okey with this, never loose your Money     4.0   
4                           Damaged product received     4.0   

                                            Comments                 Date  
0  Summer and air cooler goes synonymously. As te...  🇮🇳 on 17 March 2023  
1  Overall product is great , at this price you c...  🇮🇳 on 19 March 2023  
2                                          Read more  🇮🇳 on 19 March 2023  
3                      The media could not be loa...  🇮🇳 on 15 March 2023  
4                      The media could not be loa...  🇮🇳 on 17 March 2023  


In [9]:
df.head(5)

,Title,Ratings,Comments,Date
0,"Platina on, Garmi gone! Efficient cooler for n...",4.0,Summer and air cooler goes synonymously. As te...,🇮🇳 on 17 March 2023
1,Noise level,4.0,"Overall product is great , at this price you c...",🇮🇳 on 19 March 2023
2,Worthy for value,4.0,Read more,🇮🇳 on 19 March 2023
3,"Im okey with this, never loose your Money",4.0,The media could not be loa...,🇮🇳 on 15 March 2023
4,Damaged product received,4.0,The media could not be loa...,🇮🇳 on 17 March 2023


In [19]:
df['Date'] = pd.to_datetime(df['Date'])
df['Ratings'] = df['Ratings'].astype(float)
df.head(2)

KeyError: 'Date'

## Text Cleaning

1. lower the text
2. tokenize the text (split the text into words) and remove the punctuation
3. remove useless words that contain numbers
4. remove useless stop words like ‘the’, ‘a’ ,’this’ etc.
5. Part-Of-Speech (POS) tagging: assign a tag to every word to define 6. if it corresponds to a noun, a verb etc. using the WordNet lexical database
7. lemmatize the text: transform every word into their root form (e.g. rooms -> room, slept -> sleep)

In [ ]:
df.head(2)

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

import string
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')

In [ ]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [ ]:
# clean text data
df["Comments"] = df["Comments"].apply(lambda x: clean_text(x))

In [ ]:
df['Title'] = df['Title'].astype(str)
df['Title'] = df['Title'].apply(lambda x: clean_text(x))

In [ ]:
df.head(5)

## Feature Engineering

In [ ]:
#  add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
df["sentiments"] = df["Comments"].apply(lambda x: sid.polarity_scores(x))
df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)
'''
sid = SentimentIntensityAnalyzer()
df["sentiments_title"] = df["Title"].apply(lambda x: sid.polarity_scores(x))
df = pd.concat([df.drop(['sentiments_title'], axis=1), df['sentiments_title'].apply(pd.Series)], axis=1)
'''
df

In [ ]:
# add number of characters column
df["nb_chars"] = df["Comments"].apply(lambda x: len(x))

# add number of words column
df["nb_words"] = df["Comments"].apply(lambda x: len(x.split(" ")))

''''
# add number of characters column
df["nb_chars_title"] = df["Title"].apply(lambda x: len(x))

# add number of words column
df["nb_words_title"] = df["Title"].apply(lambda x: len(x.split(" ")))
'''

In [ ]:
# create doc2vec vector columns
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df["Comments"].apply(lambda x: x.split(" ")))]

# train a Doc2Vec model with our text data
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# transform each Comment into a vector data
doc2vec_df = df["Comments"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
df = pd.concat([df, doc2vec_df], axis=1)

'''
# transform each Title into a vector data
doc2vec_df_title = df["Title"].apply(lambda x: model.infer_vector(x.split(" "))).apply(pd.Series)
doc2vec_df_title.columns = ["doc2vec_vector_" + str(x) for x in doc2vec_df.columns]
df = pd.concat([df, doc2vec_df_title], axis=1)
'''

In [ ]:
# add tf-idfs columns
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 10)
tfidf_result = tfidf.fit_transform(df["Comments"]).toarray()
tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = df.index
df = pd.concat([df, tfidf_df], axis=1)

'''
##TF-IDF for Titles
tfidf = TfidfVectorizer(min_df = 10)
tfidf_result = tfidf.fit_transform(df["Title"]).toarray()
tfidf_df_title = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
tfidf_df_title.columns = ["word_" + str(x) for x in tfidf_df_title.columns]
tfidf_df_title.index = df.index
df = pd.concat([df, tfidf_df_title], axis=1)
'''

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 200,
        max_font_size = 40, 
        scale = 3,
        random_state = 42
    ).generate(str(data))

    fig = plt.figure(1, figsize = (20, 20))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(df["Comments"])

In [ ]:
# print wordcloud
show_wordcloud(df["Title"])

In [ ]:
df.head(2)

In [ ]:
# highest positive sentiment reviews (with more than 5 words)
df[df["nb_words"] >= 5].sort_values("pos", ascending = False)[["Comments", "pos"]].head(10)

In [ ]:
# lowest negative sentiment reviews (with more than 5 words)
df[df["nb_words"] >= 5].sort_values("neg", ascending = False)[["Comments", "neg"]].head(10)

In [ ]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df.head(2)

In [ ]:
df_recent = df[(df['Year']== 2020) & (df['Month'] != 8)]
df_recent.head(2)